In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import math
torch.manual_seed(42)


# Encoder

## embedding

## MultiHeadAttention

In [24]:
d_model = 4 # 模型维度
nhead = 2 # 多头数量
batch_size = 2 
seq_len = 3 # 长度
input = torch.rand(batch_size, seq_len,d_model)

In [25]:
class MultiHeadAttention(nn.Module):

    def __init__(self,n_head,d_model,dropout=0.1):
        super(MultiHeadAttention,self).__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.n_dim = d_model // n_head
        assert self.d_model == self.n_head * self.n_dim

        self.q_proj = nn.Linear(d_model,d_model)
        self.k_proj = nn.Linear(d_model,d_model)
        self.v_proj = nn.Linear(d_model,d_model)
        self.output_proj = nn.Linear(d_model,d_model)


        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)

    def forward(self,q,k,v):
        batch_size, seq_len, dimension = q.shape
        q = self.q_proj(q)
        k = self.k_proj(k)
        v = self.v_proj(v)

        q = q.view(batch_size, seq_len, self.n_head, self.n_dim).permute(0, 2, 1, 3)
        k = k.view(batch_size, seq_len, self.n_head, self.n_dim).permute(0, 2, 1, 3)
        v = v.view(batch_size, seq_len, self.n_head, self.n_dim).permute(0, 2, 1, 3)

        attention = self.softmax(q @ k.transpose(-1,-2) / math.sqrt(self.n_dim))

        attention = self.dropout(attention)

        output = (attention @ v).permute(0,2,1,3).contiguous().view(batch_size,seq_len,dimension)
        output = self.output_proj(output)
        return output

In [27]:
attention = MultiHeadAttention(n_head=nhead, d_model=d_model, dropout=0)

In [32]:
output = attention(input, input, input)
output

tensor([[[-0.8874, -0.5777,  0.5994, -0.5397],
         [-0.8842, -0.5756,  0.5989, -0.5419],
         [-0.8879, -0.5780,  0.5994, -0.5392]],

        [[-0.7352, -0.4972,  0.5359, -0.4731],
         [-0.7308, -0.4902,  0.5311, -0.4723],
         [-0.7295, -0.4893,  0.5307, -0.4725]]], grad_fn=<ViewBackward0>)